In [1]:
%pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [24]:
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd

# After loading your data (assuming you have X and y from your training script)
print("Original class distribution:")
print(pd.Series(y).value_counts())

# Option 1: Use SMOTE to balance classes
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

print("\nBalanced class distribution:")
print(pd.Series(y_balanced).value_counts())

# Now retrain your classifier with balanced data
clf = StressClassifier()  # Your existing classifier class
clf.train(X_balanced, y_balanced)
clf.save_model()

Original class distribution:
0.0    1400
2.0     539
1.0     415
Name: count, dtype: int64

Balanced class distribution:
0.0    1400
2.0    1400
1.0    1400
Name: count, dtype: int64

EVALUATION
              precision    recall  f1-score   support

         Low       0.50      0.44      0.47       280
    Moderate       0.53      0.60      0.57       280
        High       0.55      0.55      0.55       280

    accuracy                           0.53       840
   macro avg       0.53      0.53      0.53       840
weighted avg       0.53      0.53      0.53       840


Confusion Matrix:
[[123  82  75]
 [ 62 169  49]
 [ 61  65 154]]

ROC-AUC: 0.722

Feature importance:
  sdnn: 0.463
  rmssd: 0.537

Saved confusion.png

Saved stress_model.pkl and scaler.pkl


In [6]:
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import joblib
from datetime import datetime
from pathlib import Path
import glob
import pickle
from scipy import signal

class SamsungDataLoader:
    
    @staticmethod
    def load_all_samsung_files(data_directory):
        # Search recursively for HRV files
        hrv_files = glob.glob(data_directory + '/com.samsung.health.hrv/**/*.json', recursive=True)
        
        # Search recursively for stress files (shealth NOT health)
        stress_files = glob.glob(data_directory + '/com.samsung.shealth.stress/**/*.json', recursive=True)
        
        hr_files = []
        
        print(f"Found:")
        print(f"  Stress files: {len(stress_files)}")
        print(f"  HRV files: {len(hrv_files)}")
        
        stress_data = []
        for file in stress_files:
            try:
                with open(file, 'r') as f:
                    data = json.load(f)
                    if isinstance(data, list):
                        stress_data.extend(data)
                    else:
                        stress_data.append(data)
            except Exception as e:
                print(f"Error: {e}")
        
        hrv_data = []
        for file in hrv_files:
            if 'stress' in file.lower() or 'heart_rate' in file.lower():
                continue
            try:
                with open(file, 'r') as f:
                    data = json.load(f)
                    if isinstance(data, list):
                        hrv_data.extend(data)
                    else:
                        hrv_data.append(data)
            except Exception as e:
                print(f"Error: {e}")
        
        return stress_data, hrv_data, hr_files
    
    @staticmethod
    def process_stress_data(stress_data):
        df = pd.DataFrame(stress_data)
        df['timestamp'] = pd.to_datetime(df['start_time'], unit='ms')
        
        df['stress_normalized'] = df['score'] / df['score'].max()
        
        # split into 3 classes
        low_thresh = df['score'].quantile(0.33)
        high_thresh = df['score'].quantile(0.66)
        
        df['stress_class'] = 1  # default moderate
        df.loc[df['score'] <= low_thresh, 'stress_class'] = 0  # low
        df.loc[df['score'] > high_thresh, 'stress_class'] = 2  # high
        
        print(f"Stress data:")
        print(f"  Samples: {len(df)}")
        print(f"  Score range: {df['score'].min():.1f} - {df['score'].max():.1f}")
        print(f"  Low stress: {(df['stress_class'] == 0).sum()}")
        print(f"  Moderate stress: {(df['stress_class'] == 1).sum()}")
        print(f"  High stress: {(df['stress_class'] == 2).sum()}")
        
        return df[['timestamp', 'score', 'stress_class', 'stress_normalized']]
    
    @staticmethod
    def process_hrv_data(hrv_data):
        df = pd.DataFrame(hrv_data)
        df['timestamp'] = pd.to_datetime(df['start_time'], unit='ms')
        
        if 'sdnn' not in df.columns:
            df['sdnn'] = np.nan
        if 'rmssd' not in df.columns:
            df['rmssd'] = np.nan
        
        print(f"HRV data:")
        print(f"  Samples: {len(df)}")
        print(f"  SDNN range: {df['sdnn'].min():.1f} - {df['sdnn'].max():.1f}")
        print(f"  RMSSD range: {df['rmssd'].min():.1f} - {df['rmssd'].max():.1f}")
        
        return df[['timestamp', 'sdnn', 'rmssd']]
    
    @staticmethod
    def merge_stress_and_hrv(df_stress, df_hrv):
        df_stress['timestamp_rounded'] = df_stress['timestamp'].dt.round('1min')
        df_hrv['timestamp_rounded'] = df_hrv['timestamp'].dt.round('1min')
        
        merged = pd.merge_asof(
            df_hrv.sort_values('timestamp'),
            df_stress.sort_values('timestamp'),
            on='timestamp',
            direction='nearest',
            tolerance=pd.Timedelta('1hr')
        )
        
        merged = merged.dropna(subset=['stress_class'])
        merged = merged.dropna(subset=['sdnn', 'rmssd'])
        
        print(f"Merged data:")
        print(f"  Matched samples: {len(merged)}")
        print(f"  Time span: {merged['timestamp'].min()} to {merged['timestamp'].max()}")
        print(f"  Duration: {(merged['timestamp'].max() - merged['timestamp'].min()).days} days")
        
        return merged[['timestamp', 'sdnn', 'rmssd', 'stress_class', 'score']]


class WESADDataLoader:
    
    @staticmethod
    def calculate_hrv(rr_intervals):
        """Calculate HRV stuff"""
        if len(rr_intervals) < 2:
            return None, None
        
        # clean the data a bit
        rr_intervals = rr_intervals[(rr_intervals > 300) & (rr_intervals < 2000)]
        
        if len(rr_intervals) < 2:
            return None, None
        
        sdnn = np.std(rr_intervals)
        
        diff_rr = np.diff(rr_intervals)
        rmssd = np.sqrt(np.mean(diff_rr ** 2))
        
        return sdnn, rmssd
    
    @staticmethod
    def get_rr_from_ecg(ecg_signal, fs=700):
        """Extract RR intervals"""
        # filter the ecg
        nyq = fs / 2
        low = 0.5 / nyq
        high = 40 / nyq
        b, a = signal.butter(4, [low, high], btype='band')
        ecg_filt = signal.filtfilt(b, a, ecg_signal)
        
        # normalize
        ecg_norm = (ecg_filt - np.mean(ecg_filt)) / np.std(ecg_filt)
        
        # find peaks
        min_dist = int(0.4 * fs)
        thresh = np.mean(ecg_norm) + 0.5 * np.std(ecg_norm)
        peaks = signal.find_peaks(ecg_norm, height=thresh, distance=min_dist)[0]
        
        if len(peaks) < 2:
            return None
        
        # get RR intervals in ms
        rr = np.diff(peaks) / fs * 1000
        
        return rr
    
    @staticmethod
    def load_wesad_subject(subject_path):
        """Load one subject"""
        with open(subject_path, 'rb') as f:
            data = pickle.load(f, encoding='latin1')
        
        # get chest data
        if 'signal' not in data or 'chest' not in data['signal']:
            print(f"No chest data in {subject_path}")
            return None
        
        chest = data['signal']['chest']
        labels = data['label']
        
        # get ecg - usually first channel
        if isinstance(chest, dict):
            ecg = chest['ECG'].flatten() if 'ECG' in chest else None
        else:
            ecg = chest[:, 0] if len(chest.shape) > 1 else chest
            
        if ecg is None:
            return None
        
        fs = 700  # sampling rate
        
        return ecg, labels, fs
    
    @staticmethod
    def process_wesad_data(wesad_directory):
        """Process WESAD subjects"""
        subject_files = glob.glob(f"{wesad_directory}/**/S*.pkl", recursive=True)
        
        print(f"\nFound {len(subject_files)} WESAD subjects")
        
        features = []
        
        for subj_file in subject_files:
            subj = Path(subj_file).stem
            print(f"Processing {subj}...")
            
            result = WESADDataLoader.load_wesad_subject(subj_file)
            if result is None:
                continue
            
            ecg, labels, fs = result
            
            # process in 60 sec windows
            win_size = 60 * fs
            step = win_size // 2  # overlap
            
            for i in range((len(ecg) - win_size) // step + 1):
                start = i * step
                end = start + win_size
                
                if end > len(ecg):
                    break
                
                ecg_win = ecg[start:end]
                label_win = labels[start:end]
                
                # get most common label
                vals, counts = np.unique(label_win, return_counts=True)
                label = vals[np.argmax(counts)]
                
                # WESAD labels: 0=undefined, 1=baseline, 2=stress, 3=amusement, 4=meditation
                # map to 3 classes: 0=low, 1=moderate, 2=high
                if label == 0:
                    continue
                elif label == 1:  # baseline
                    stress = 0
                elif label == 2:  # stress
                    stress = 2
                elif label == 3:  # amusement
                    stress = 1
                elif label == 4:  # meditation  
                    stress = 0
                else:
                    continue
                
                # get RR intervals
                rr = WESADDataLoader.get_rr_from_ecg(ecg_win, fs)
                
                if rr is None or len(rr) < 10:
                    continue
                
                # calc HRV
                sdnn, rmssd = WESADDataLoader.calculate_hrv(rr)
                
                if sdnn is None or rmssd is None:
                    continue
                
                features.append({
                    'subject': subj,
                    'sdnn': sdnn,
                    'rmssd': rmssd,
                    'stress_class': stress,
                    'source': 'WESAD'
                })
        
        df = pd.DataFrame(features)
        
        print(f"\nWESAD data:")
        print(f"  Total samples: {len(df)}")
        print(f"  Low: {(df['stress_class'] == 0).sum()}")
        print(f"  Moderate: {(df['stress_class'] == 1).sum()}")
        print(f"  High: {(df['stress_class'] == 2).sum()}")
        print(f"  SDNN range: {df['sdnn'].min():.1f} - {df['sdnn'].max():.1f}")
        print(f"  RMSSD range: {df['rmssd'].min():.1f} - {df['rmssd'].max():.1f}")
        
        return df


class StressClassifier:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = None
        self.features = ['sdnn', 'rmssd']
    
    def train(self, X, y, test_size=0.2):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )
        
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # xgboost for 3 classes
        self.model = xgb.XGBClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            num_class=3
        )
        
        self.model.fit(X_train_scaled, y_train, verbose=False)
        
        y_pred = self.model.predict(X_test_scaled)
        y_proba = self.model.predict_proba(X_test_scaled)
        
        print("\nEVALUATION")
        print("="*50)
        
        labels = ['Low', 'Moderate', 'High']
        print(classification_report(y_test, y_pred, target_names=labels))
        
        print(f"\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        
        # try to get ROC-AUC
        try:
            auc = roc_auc_score(y_test, y_proba, multi_class='ovr', average='macro')
            print(f"\nROC-AUC: {auc:.3f}")
        except:
            print("\nCouldn't compute ROC-AUC")
        
        print("\nFeature importance:")
        for f, imp in zip(self.features, self.model.feature_importances_):
            print(f"  {f}: {imp:.3f}")
        
        # plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, cmap='Blues')
        plt.colorbar()
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.xticks([0,1,2], labels)
        plt.yticks([0,1,2], labels)
        
        # add numbers
        for i in range(3):
            for j in range(3):
                plt.text(j, i, cm[i,j], ha='center', va='center')
        
        plt.savefig('confusion.png', dpi=150)
        plt.close()
        print("\nSaved confusion.png")
        
        return self.model
    
    def predict(self, X_new):
        if self.model is None:
            raise ValueError("Model not trained!")
        X_scaled = self.scaler.transform(X_new)
        preds = self.model.predict(X_scaled)
        proba = self.model.predict_proba(X_scaled)
        return preds, proba
    
    def save_model(self, model_file='stress_model.pkl', scaler_file='scaler.pkl'):
        joblib.dump(self.model, model_file)
        joblib.dump(self.scaler, scaler_file)
        print(f"\nSaved {model_file} and {scaler_file}")


if __name__ == '__main__':
    print("Stress Classification - 3 Classes")
    print("="*50)
    
    # paths
    samsung_dir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\jsons'
    wesad_dir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\WESAD'
    
    use_samsung = True
    use_wesad = True  # set False if no WESAD
    
    all_data = []
    
    # load samsung
    if use_samsung:
        print("\nLoading Samsung data...")
        try:
            loader = SamsungDataLoader()
            stress_data, hrv_data, hr_data = loader.load_all_samsung_files(samsung_dir)
            
            if stress_data and hrv_data:
                df_stress = loader.process_stress_data(stress_data)
                df_hrv = loader.process_hrv_data(hrv_data)
                df_samsung = loader.merge_stress_and_hrv(df_stress, df_hrv)
                
                df_samsung['source'] = 'Samsung'
                all_data.append(df_samsung[['sdnn', 'rmssd', 'stress_class', 'source']])
                print(f"Added {len(df_samsung)} Samsung samples")
            else:
                print("No Samsung data!")
        except Exception as e:
            print(f"Samsung error: {e}")
    
    # load wesad
    if use_wesad:
        print("\nLoading WESAD data...")
        try:
            wesad_loader = WESADDataLoader()
            df_wesad = wesad_loader.process_wesad_data(wesad_dir)
            
            if df_wesad is not None and len(df_wesad) > 0:
                all_data.append(df_wesad[['sdnn', 'rmssd', 'stress_class', 'source']])
            else:
                print("No WESAD data!")
        except Exception as e:
            print(f"WESAD error: {e}")
    
    if not all_data:
        print("\nERROR: No data!")
        exit(1)
    
    # combine
    print("\nCombining datasets...")
    df = pd.concat(all_data, ignore_index=True)
    
    print(f"\nTotal samples: {len(df)}")
    for src in df['source'].unique():
        src_df = df[df['source'] == src]
        print(f"\n{src}:")
        print(f"  Samples: {len(src_df)}")
        print(f"  Low: {(src_df['stress_class'] == 0).sum()}")
        print(f"  Moderate: {(src_df['stress_class'] == 1).sum()}")
        print(f"  High: {(src_df['stress_class'] == 2).sum()}")
    
    # train
    print("\nTraining model...")
    X = df[['sdnn', 'rmssd']].values
    y = df['stress_class'].values
    
    clf = StressClassifier()
    clf.train(X, y)
    clf.save_model()
    
    print("\nDONE!")

Stress Classification - 3 Classes

Loading Samsung data...
Found:
  Stress files: 119
  HRV files: 35
Stress data:
  Samples: 5091
  Score range: 0.0 - 100.0
  Low stress: 1711
  Moderate stress: 1677
  High stress: 1703
HRV data:
  Samples: 2807
  SDNN range: 30.8 - 174.1
  RMSSD range: 25.6 - 133.1
Merged data:
  Matched samples: 855
  Time span: 2025-12-26 06:48:02.825000 to 2026-01-02 00:36:18.909000
  Duration: 6 days
Added 855 Samsung samples

Loading WESAD data...

Found 15 WESAD subjects
Processing S10...
Processing S11...
Processing S13...
Processing S14...
Processing S15...
Processing S16...
Processing S17...
Processing S2...
Processing S3...
Processing S4...
Processing S5...
Processing S6...
Processing S7...
Processing S8...
Processing S9...

WESAD data:
  Total samples: 1499
  Low: 981
  Moderate: 186
  High: 332
  SDNN range: 10.4 - 256.0
  RMSSD range: 3.6 - 271.4

Combining datasets...

Total samples: 2354

Samsung:
  Samples: 855
  Low: 419
  Moderate: 229
  High: 207



In [25]:
import joblib
import numpy as np

model = joblib.load('stress_model.pkl')
scaler = joblib.load('scaler.pkl')

# Test with EXTREME low HRV (should be high stress)
test_low_hrv = np.array([[20, 15]])  # Very low SDNN and RMSSD
scaled = scaler.transform(test_low_hrv)
pred = model.predict(scaled)
proba = model.predict_proba(scaled)

print(f"Low HRV prediction: {pred[0]}")  # Should be 2 (high stress)
print(f"Probabilities: Low={proba[0][0]:.2f}, Mod={proba[0][1]:.2f}, High={proba[0][2]:.2f}")

# Test with high HRV (should be low stress)
test_high_hrv = np.array([[150, 200]])
scaled = scaler.transform(test_high_hrv)
pred = model.predict(scaled)
proba = model.predict_proba(scaled)

print(f"High HRV prediction: {pred[0]}")  # Should be 0 (low stress)
print(f"Probabilities: Low={proba[0][0]:.2f}, Mod={proba[0][1]:.2f}, High={proba[0][2]:.2f}")

Low HRV prediction: 2
Probabilities: Low=0.25, Mod=0.02, High=0.73
High HRV prediction: 0
Probabilities: Low=0.57, Mod=0.42, High=0.01
